In [1]:
import os

from typing import List

import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.cluster import KMeans
from folium import Map, CircleMarker, PolyLine, FeatureGroup
from shapely import MultiPoint, centroid
from itertools import permutations, product, combinations, chain

from formation.helpers.map import calcul_distances, calcul_distance

import plotly.io as pio

pio.get_chrome()

2025-10-16 23:40:49.561 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


PosixPath('/home/jaden/Sources/followchon_back/.venv/lib/python3.12/site-packages/choreographer/cli/browser_exe/chrome-linux64/chrome')

In [2]:
COLORS = px.colors.qualitative.Vivid + px.colors.qualitative.Alphabet

In [3]:
def calcul_centroids(df_cluster_points: pd.DataFrame):
    clusters = df_cluster_points['cluster'].unique()

    df_centroids = pd.DataFrame()

    for i in clusters:
        cluster_points = df_cluster_points[df_cluster_points['cluster'] == str(i)].loc[:, ['lat', 'lng']].to_numpy()
        c = centroid(MultiPoint(cluster_points))

        df_centroids = pd.concat([
            df_centroids,
            pd.DataFrame({
                'index': [i],
                'lat': [c.x],
                'lng': [c.y]
            })
        ], ignore_index=True)

    df_centroids['index'] = df_centroids['index'].astype(str)
    df_centroids['lat'] = df_centroids['lat'].astype(float)
    df_centroids['lng'] = df_centroids['lng'].astype(float)

    return calcul_distances(df_centroids.set_index('index'))

def draw_map(df_points, end=False):
    m = Map(
        location=(46.227638, 2.213749),
        zoom_start=6,
    )

    fg = FeatureGroup(name="Markers")

    for i, point in df_points.iterrows():
        if 'lat' in point and 'lng' in point:
            fg.add_child(
                CircleMarker(
                    location=[
                        point['lat'],
                        point['lng'],
                    ],
                    tooltip=f"{point['index']} ({point['cluster']})",
                    fill=True,
                    #color=COLORS[int(point['cluster'])],
                    weight=0,
                    fill_opacity=0.6,
                    radius=15,
                )
            )

        if 'center_lat' in point and 'center_lng' in point:
            fg.add_child(
                CircleMarker(
                    location=[
                        point['center_lat'],
                        point['center_lng'],
                    ],
                    #tooltip=f"{point['index']} ({point['cluster_current']})",
                    fill=True,
                    #color=COLORS[int(point['cluster_current'])],
                    weight=0,
                    fill_opacity=0.6,
                    radius=15,
                )
            )

        if 'lat_next' in point and 'lng_next' in point:
            fg.add_child(
                PolyLine(
                    locations=[
                        [point['lat'], point['lng']],
                        [point['lat_next'], point['lng_next']],
                    ],
                    color="black",
                    weight=4,
                )
            )

    m.add_child(fg)

    return m

In [4]:
def best_path(df_centroids: pd.DataFrame, df_points: pd.DataFrame, df_points_distances: pd.DataFrame) -> pd.DataFrame | None:
    cluster_indexes = list(df_centroids['index'].unique())
    cluster_count = len(cluster_indexes)

    print(f"- [Step 1] with {cluster_count} clusters")
    tours_indexes = [
        (
            cluster_indexes[i],
            cluster_indexes[(i + 1) % cluster_count],
            cluster_indexes[(i + 2) % cluster_count]
        )
        for i in range(cluster_count)
    ]

    tours_dict = {}

    for (cluster_prev, cluster_current, cluster_next) in tours_indexes:
        # print("-- Step :", (cluster_prev, cluster_current, cluster_next))

        bridge = (cluster_prev, cluster_current, cluster_next)

        tours_dict[bridge] = []

        df_cluster_current = df_points[df_points['cluster'] == cluster_current]
        df_cluster_prev = df_points[df_points['cluster'] == cluster_prev]
        df_cluster_next = df_points[df_points['cluster'] == cluster_next]

        bridges_prev = list(
            product(
                df_cluster_prev['index'].unique(),
                df_cluster_current['index'].unique(),
            )
        )

        bridges_next = list(
            product(
                df_cluster_current['index'].unique(),
                df_cluster_next['index'].unique(),
            )
        )

        bridge_paths = list(
            product(bridges_prev, bridges_next)
        )

        for ((cluster_prev_in, cluster_current_in), (cluster_current_out, cluster_next_out)) in bridge_paths:
            if cluster_current_in != cluster_current_out or df_cluster_current.shape[0] == 1:
                if df_cluster_current.shape[0] == 1 and cluster_current_in == cluster_current_out:
                    cluster_path = [cluster_current_in]
                    if cluster_path not in tours_dict[bridge]:
                        # print("--- Path :", cluster_path)
                        tours_dict[bridge].append(cluster_path)
                else:
                    cluster_between = df_cluster_current[
                        (df_cluster_current['index'] != cluster_current_in) &
                        (df_cluster_current['index'] != cluster_current_out)
                        ]['index'].values

                    for between in list(permutations(cluster_between)):
                        cluster_path = [cluster_current_in] + list(between) + [cluster_current_out]

                        if cluster_path not in tours_dict[bridge]:
                            # print("--- Path :", cluster_path)
                            tours_dict[bridge].append(cluster_path)

    values = list(tours_dict.values())
    paths_pairs = []
    paths = []

    combinations_list = list(product(*values))

    print(f"- [Step 2] with {len(combinations_list)} combinaisons")

    if len(combinations_list) > calcul_max_combinations:
        return None

    for combination in combinations_list:
        path = []
        for step in combination:
            path = path + step

        pairs = list(zip(path[:-1], path[1:]))
        pairs.append((path[-1], path[0]))

        paths.append(path)
        paths_pairs.append(pairs)

    print(f"- [Step 3]")

    distances = dict()
    for i, path in enumerate(paths_pairs):
        distance = 0
        for point_a, point_b in path:
            distance += df_points_distances[point_a][point_b]

        distances[tuple(path)] = distance

    tours_best_distance = min(distances.values())
    tours_best_pairs = min(distances, key=distances.get)

    points_prev, points_next = zip(*tours_best_pairs)

    return pd.DataFrame({
        'point_prev': list(points_prev),
        'point_next': list(points_next),
    }).merge(
        df_points,
        left_on='point_prev',
        right_on='index',
        how='left',
    ).merge(
        df_points,
        left_on='point_next',
        right_on='index',
        how='left',
        suffixes=('', '_next'),
    ).drop(columns=['point_prev', 'point_next'])

def calc_next(df_points: pd.DataFrame, df_points_distances: pd.DataFrame) -> pd.DataFrame:
    df_points['index_next'] = df_points['index'].shift(periods=-1, fill_value='')
    df_points['index_next'] = df_points['index_next'].astype(str)
    df_points.loc[df_points.index[-1], 'index_next'] = df_points.iloc[0]['index']

    df_points['cluster_next'] = df_points['cluster'].shift(periods=-1, fill_value='')
    df_points.loc[df_points.index[-1], 'cluster_next'] = df_points.iloc[0]['cluster']

    df_points['lat_next'] = df_points['lat'].shift(periods=-1, fill_value='')
    df_points.loc[df_points.index[-1], 'lat_next'] = df_points.iloc[0]['lat']

    df_points['lng_next'] = df_points['lng'].shift(periods=-1, fill_value='')
    df_points.loc[df_points.index[-1], 'lng_next'] = df_points.iloc[0]['lng']

    df_points['x_next'] = df_points['x'].shift(periods=-1, fill_value='')
    df_points.loc[df_points.index[-1], 'x_next'] = df_points.iloc[0]['x']

    df_points['y_neyt'] = df_points['y'].shift(periods=-1, fill_value='')
    df_points.loc[df_points.index[-1], 'y_neyt'] = df_points.iloc[0]['y']

    df_points['distance'] = df_points.apply(lambda x: df_points_distances[x['index']][x['index_next']], axis=1)

    return df_points

In [5]:
def simple_path(df_centroids: pd.DataFrame, df_points: pd.DataFrame, df_points_distances: pd.DataFrame) -> pd.DataFrame:
    centroids_order = df_centroids['index'].to_dict()
    centroids_order = {v: k for k, v in centroids_order.items()}

    df_points['order'] = df_points['cluster'].apply(lambda x: centroids_order[x])
    df_points = df_points.sort_values('order').reset_index(drop=True).drop(columns=['order'])

    df_points = calc_next(df_points, df_points_distances)

    distance_total = df_points['distance'].sum()
    cluster_prev = None
    index_prev = None

    print(f'- [Step 3] with {df_points.shape[0]} points and distance min {distance_total}')

    for index_current, point in df_points.iterrows():
        # if cluster_prev is not None and index_prev is not None and point['cluster'] == cluster_prev:
        if index_prev is not None and index_current > index_prev:
            # print(f'-- Cluster {cluster_prev} with distance min {distance_total}')

            df_points_copy = df_points.copy()
            df_points_copy.iloc[[index_current, index_prev]] = df_points_copy.iloc[[index_prev, index_current]].values

            df_points_copy = calc_next(df_points_copy, df_points_distances)

            distance_total_new = df_points_copy['distance'].sum()
            if distance_total_new < distance_total:
                distance_total = distance_total_new
                df_points = df_points_copy
        # else:
        # cluster_prev = str(point['cluster'])
        # index_prev = index_current

        index_prev = index_current

    return df_points

In [6]:
df_cities = pd.read_csv(
    '../data/formated/cities-france.csv',
    dtype={'code_insee': str, 'dep_code': str, 'nom_standard': str, 'latitude_mairie': float, 'longitude_mairie': float},
).rename(columns={'code_insee': 'index', 'nom_standard': 'name', 'latitude_mairie': 'lat', 'longitude_mairie': 'lng'}) \
    .set_index('index') \
    .loc[:, ['lat', 'lng']]

In [7]:
city_count = 70
cluster_max_length = 3
cluster_min_length = 3
calcul_max_combinations = 40000 #30000 #20000
calcul_max_clusters = 40 #30 #20

df_points_coords, df_points_distances = calcul_distances(df_cities.sample(city_count))

if not os.path.exists('data'):
    os.makedirs('data')

if not os.path.exists('export'):
    os.makedirs('export')

steps_count = 1

while True:
    print(f'Step {steps_count} with {df_points_coords.shape[0]} points')

    points = df_points_coords.loc[:,['x', 'y']].to_numpy()

    k_kmeans = 1
    points_by_cluster = None

    if df_points_coords.shape[0] > cluster_min_length:
        if cluster_max_length ** 2 > df_points_coords.shape[0]:
            k_kmeans = cluster_min_length
        else:
            k_kmeans = df_points_coords.shape[0] // cluster_max_length

    while points_by_cluster is None or points_by_cluster > cluster_max_length:
        model_kmeans = KMeans(n_clusters=k_kmeans)
        model_kmeans.fit(points)
        df_points_coords['cluster'] = model_kmeans.predict(points)
        df_points_coords['cluster'] = df_points_coords['cluster'].astype(str)

        points_by_cluster = df_points_coords[['cluster', 'index']].groupby('cluster').count().max().values[0]

        print(f'- with {k_kmeans} clusters with {points_by_cluster} max size')

        if points_by_cluster > cluster_max_length:
            k_kmeans += 1

    draw_map(df_points_coords.reset_index()).save(f'export/step_{steps_count}_map.html')

    df_points_coords.to_csv(f'data/step_{steps_count}_points.csv', index=False)
    df_points_distances.to_csv(f'data/step_{steps_count}_distances.csv')

    fig = px.scatter(
        df_points_coords.reset_index(),
        x='x',
        y='y',
        color='cluster',
        hover_name='index',
        height=700,
        size=[1] * df_points_coords.shape[0],
    )
    fig.write_image(f"export/step_{steps_count}_points.jpg")

    if df_points_coords.shape[0] > cluster_min_length:
        df_centroids_coords, df_centroids_distances = calcul_centroids(df_points_coords)

        df_points_coords, df_points_distances = df_centroids_coords, df_centroids_distances

        steps_count += 1
    else:
        break

/home/jaden/Sources/followchon_back/.venv/lib/python3.12/site-packages/sklearn/manifold/_mds.py:677: FutureWarning:

The default value of `n_init` will change from 4 to 1 in 1.9.



Step 1 with 70 points
- with 23 clusters with 9 max size
- with 24 clusters with 6 max size
- with 25 clusters with 7 max size
- with 26 clusters with 6 max size
- with 27 clusters with 7 max size
- with 28 clusters with 6 max size
- with 29 clusters with 6 max size
- with 30 clusters with 5 max size
- with 31 clusters with 6 max size
- with 32 clusters with 6 max size
- with 33 clusters with 6 max size
- with 34 clusters with 5 max size
- with 35 clusters with 7 max size
- with 36 clusters with 6 max size
- with 37 clusters with 5 max size
- with 38 clusters with 6 max size
- with 39 clusters with 6 max size
- with 40 clusters with 5 max size
- with 41 clusters with 4 max size
- with 42 clusters with 4 max size
- with 43 clusters with 6 max size
- with 44 clusters with 5 max size
- with 45 clusters with 4 max size
- with 46 clusters with 4 max size
- with 47 clusters with 4 max size
- with 48 clusters with 5 max size
- with 49 clusters with 4 max size
- with 50 clusters with 4 max siz

/home/jaden/Sources/followchon_back/.venv/lib/python3.12/site-packages/sklearn/manifold/_mds.py:677: FutureWarning:

The default value of `n_init` will change from 4 to 1 in 1.9.



Step 2 with 53 points
- with 17 clusters with 6 max size
- with 18 clusters with 7 max size
- with 19 clusters with 6 max size
- with 20 clusters with 5 max size
- with 21 clusters with 4 max size
- with 22 clusters with 5 max size
- with 23 clusters with 4 max size
- with 24 clusters with 4 max size
- with 25 clusters with 4 max size
- with 26 clusters with 4 max size
- with 27 clusters with 4 max size
- with 28 clusters with 5 max size
- with 29 clusters with 5 max size
- with 30 clusters with 4 max size
- with 31 clusters with 4 max size
- with 32 clusters with 4 max size
- with 33 clusters with 4 max size
- with 34 clusters with 4 max size
- with 35 clusters with 3 max size


/home/jaden/Sources/followchon_back/.venv/lib/python3.12/site-packages/sklearn/manifold/_mds.py:677: FutureWarning:

The default value of `n_init` will change from 4 to 1 in 1.9.



...

Step 4 with 18 points
- with 6 clusters with 5 max size
- with 7 clusters with 3 max size


/home/jaden/Sources/followchon_back/.venv/lib/python3.12/site-packages/sklearn/manifold/_mds.py:677: FutureWarning:

The default value of `n_init` will change from 4 to 1 in 1.9.



Step 5 with 7 points
- with 3 clusters with 3 max size


/home/jaden/Sources/followchon_back/.venv/lib/python3.12/site-packages/sklearn/manifold/_mds.py:677: FutureWarning:

The default value of `n_init` will change from 4 to 1 in 1.9.



Step 6 with 3 points
- with 1 clusters with 3 max size


In [8]:
steps_list = range(2, steps_count + 1)[::-1]
print("steps :", list(steps_list))
df_tour = pd.DataFrame()

n = steps_list[0]
df_centroids = pd.read_csv('data/step_' + str(n) + '_points.csv', dtype={'index': str, 'cluster': str})

for n in steps_list:
    df_cluster = None

    df_centroids_distances = pd.read_csv('data/step_' + str(n) + '_distances.csv', dtype={'index': str})
    df_centroids_distances = df_centroids_distances.set_index('index')

    print(f'Begin layer {n - 1} with {df_centroids.shape[0]} points')

    df_points = pd.read_csv('data/step_' + str(n - 1) + '_points.csv', dtype={'index': str, 'cluster': str})

    df_points_distances = pd.read_csv('data/step_' + str(n - 1) + '_distances.csv', dtype={'index': str})
    df_points_distances = df_points_distances.set_index('index')

    if df_centroids.shape[0] < calcul_max_clusters:
        df_cluster = best_path(df_centroids, df_points, df_points_distances)

    if df_cluster is None:
        df_cluster = simple_path(df_centroids, df_points, df_points_distances)

    draw_map(df_cluster, True).save(f'export/step_{n - 1}_map.html')
    df_centroids = df_cluster[['index', 'lat', 'lng', 'x', 'y', 'cluster']]

    print(f'End layer {n - 1} with {df_centroids.shape[0]} points')

steps : [6, 5, 4, 3, 2]
Begin layer 5 with 3 points
- [Step 1] with 3 clusters
- [Step 2] with 24 combinaisons
- [Step 3]
End layer 5 with 7 points
Begin layer 4 with 7 points
- [Step 1] with 7 clusters
- [Step 2] with 15552 combinaisons
- [Step 3]
End layer 4 with 18 points
Begin layer 3 with 18 points
- [Step 1] with 18 clusters
- [Step 2] with 1492992 combinaisons
- [Step 3] with 35 points and distance min 5464.0
End layer 3 with 35 points
Begin layer 2 with 35 points
- [Step 1] with 35 clusters
- [Step 2] with 1990656 combinaisons
- [Step 3] with 53 points and distance min 5564.0
End layer 2 with 53 points
Begin layer 1 with 53 points
- [Step 3] with 70 points and distance min 5757.0
End layer 1 with 70 points
